In [3]:
import pandas as pd
import numpy  as np
from pymongo import MongoClient
from sodapy import Socrata

 <h2>1. Reading data from source and storing it into a non-relational database(MongoDB)</h2>

In [8]:
#creating the database and collection in MongoDB
from pymongo import MongoClient
client = MongoClient("mongodb://%s:%s@127.0.0.1" % ("dap", "dap"))
db = client['project_db']
collection1 = db['crashdata_people']
collection2 = db['crashdata_vehicles']
collection3 = db['crashdata_crashes']
#reading data from source
source = Socrata( "data.cityofnewyork.us",app_token='jq8rcV5T3LG4G66zxRAHySERq', username='x22111034@student.ncirl.ie', password='dap@1234', timeout=150 )
offset = 0
batch_size = 10000
result_1 = []
result_2 = []
result_3 = []
while True:
    batch_1 = source.get("bm4k-52h4", limit=batch_size, offset=offset)
    if not batch_1:
        break
    result_1.extend(batch_1)
    offset += batch_size

while True:
    batch_2 = source.get("f55k-p6yu", limit=batch_size, offset=offset)
    if not batch_2:
        break
    result_2.extend(batch_2)
    offset += batch_size
    
    
while True:
    batch_3 = source.get("h9gi-nx95", limit=batch_size, offset=offset)
    if not batch_3:
        break
    result_3.extend(batch_3)
    offset += batch_size

In [10]:
#inserting data into the non-relational database
collection1.insert_many(result_1)
#collection1.delete_many({})
collection2.insert_many(result_2)
#collection2.delete_many({})
collection3.insert_many(result_3)

In [12]:
#reading data from mongoDB and converting it into pandas dataframe
vdocs = collection1.find()
vdata = [doc for doc in vdocs]
vehicle_df = pd.DataFrame(vdata)

pdocs = collection2.find()
pdata = [doc for doc in pdocs]
people_df = pd.DataFrame(pdata)

cdocs = collection3.find()
cdata = [doc for doc in cdocs]
crash_df = pd.DataFrame(cdata)

<h2>2. Data Cleaning and Pre-processing

In [14]:
vehicle_df.columns
vehicle_df.isnull().sum()
vehicle_df_clean1 = vehicle_df.drop(['_id','unique_id','vehicle_make','vehicle_year','travel_direction','driver_license_jurisdiction','vehicle_damage', 'vehicle_damage_1','vehicle_damage_2','vehicle_damage_3','contributing_factor_2','vehicle_model','public_property_damage_type'], axis=1)

In [18]:
people_df.columns
people_df.isnull().sum()
people_df_clean1 = people_df.drop(['_id','unique_id','person_id','vehicle_id','ejection','bodily_injury','position_in_vehicle','safety_equipment', 'complaint', 'ped_location','ped_action','contributing_factor_1','contributing_factor_2'],axis=1)

In [25]:
crash_df.columns
crash_df.isnull().sum()
crash_df_clean1 = crash_df.drop(['_id','on_street_name','off_street_name','vehicle_type_code2','latitude','longitude','location','cross_street_name','contributing_factor_vehicle_2','contributing_factor_vehicle_3','contributing_factor_vehicle_4','contributing_factor_vehicle_5','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5'],axis=1)

In [86]:
vehicle_df_clean1.head()

,collision_id,crash_date,crash_time,vehicle_id,state_registration,vehicle_type,contributing_factor_1,vehicle_occupants,driver_sex,driver_license_status,pre_crash,point_of_impact,public_property_damage
0,100201,2012-09-07T00:00:00.000,9:03,1,NY,PASSENGER VEHICLE,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN
1,4213082,2019-09-23T00:00:00.000,8:15,0553ab4d-9500-4cba-8d98-f4d7f89d5856,NY,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,1,M,Licensed,Going Straight Ahead,Left Front Bumper,N
2,3307608,2015-10-02T00:00:00.000,17:18,2,NY,TAXI,Driver Inattention/Distraction,NaN,NaN,NaN,Going Straight Ahead,NaN,NaN
3,3308693,2015-10-04T00:00:00.000,20:34,1,NY,PASSENGER VEHICLE,Unspecified,NaN,NaN,NaN,Parked,NaN,NaN
4,297666,2013-04-25T00:00:00.000,21:15,1,NY,PASSENGER VEHICLE,Other Vehicular,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
people_df_clean1.head()

,collision_id,crash_date,crash_time,person_type,person_injury,ped_role,person_sex,person_age,emotional_status
0,4342381,2020-08-27T00:00:00.000,13:49,Occupant,Unspecified,Registrant,U,NaN,NaN
1,4343135,2020-08-28T00:00:00.000,18:00,Occupant,Unspecified,Driver,F,27,Does Not Apply
2,4342819,2020-08-28T00:00:00.000,21:35,Occupant,Unspecified,Registrant,U,NaN,NaN
3,4342607,2020-08-28T00:00:00.000,12:32,Occupant,Unspecified,Passenger,F,40,Does Not Apply
4,4343244,2020-08-27T00:00:00.000,19:30,Occupant,Unspecified,Registrant,U,NaN,NaN


In [28]:
inter_df = pd.merge(crash_df_clean1,vehicle_df_clean1, on='collision_id')
merged_df = pd.merge(inter_df, people_df_clean1,on='collision_id')

In [35]:
merged_df_cleaned1=merged_df.drop(['crash_date_x','crash_date_y','crash_time_x','crash_time_y','public_property_damage'],axis=1)

In [37]:
merged_df_cleaned1.columns

Index(['number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'collision_id', 'vehicle_type_code1',
       'borough', 'zip_code', 'vehicle_id', 'state_registration',
       'vehicle_type', 'contributing_factor_1', 'vehicle_occupants',
       'driver_sex', 'driver_license_status', 'pre_crash', 'point_of_impact',
       'public_property_damage', 'crash_date', 'crash_time', 'person_type',
       'person_injury', 'ped_role', 'person_sex', 'person_age',
       'emotional_status'],
      dtype='object')

In [38]:
merged_df_cleaned2 = merged_df_cleaned1.dropna()

In [55]:
merged_df_cleaned2.transpose()

,0,1,2,3,4,5,6,7,8,9,...,416699,416700,416701,416702,416703,416704,416705,416706,416707,416708
number_of_persons_injured,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,0
number_of_persons_killed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0
number_of_pedestrians_injured,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
number_of_pedestrians_killed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
number_of_cyclist_injured,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
number_of_cyclist_killed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
number_of_motorist_injured,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,0
number_of_motorist_killed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0
contributing_factor_vehicle_1,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Passing or Lane Usage Improper,Driver Inexperience,Driver Inexperience,Unspecified,Alcohol Involvement,...,Driver Inattention/Distraction,Following Too Closely,Following Too Closely,Following Too Closely,Following Too Closely,Unsafe Speed,Unsafe Speed,Unsafe Speed,Unsafe Speed,Driver Inattention/Distraction
collision_id,4456314,4457022,4457022,4457022,4457022,4456686,4456588,4456588,4456879,4455749,...,4288210,4252952,4252952,4252952,4252952,4289784,4289784,4289784,4289784,4307427


In [50]:
merged_df_cleaned2 = merged_df_cleaned2.drop(['emotional_state','pre_crash','point_of_impact'])

number_of_persons_injured        object
number_of_persons_killed         object
number_of_pedestrians_injured    object
number_of_pedestrians_killed     object
number_of_cyclist_injured        object
number_of_cyclist_killed         object
number_of_motorist_injured       object
number_of_motorist_killed        object
contributing_factor_vehicle_1    object
collision_id                     object
vehicle_type_code1               object
borough                          object
zip_code                         object
vehicle_id                       object
state_registration               object
vehicle_type                     object
contributing_factor_1            object
vehicle_occupants                object
driver_sex                       object
driver_license_status            object
pre_crash                        object
point_of_impact                  object
public_property_damage           object
crash_date                       object
crash_time                       object


Occupant           401155
Pedestrian           7442
Bicyclist            6751
Other Motorized      1361
Name: person_type, dtype: int64

In [76]:
merged_df_cleaned2['driver_license_status'].value_counts()

Licensed      401923
Unlicensed      9071
Permit          5715
Name: driver_license_status, dtype: int64

In [80]:
[merged_df_cleaned2.columns]

[Index(['number_of_persons_injured', 'number_of_persons_killed',
        'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
        'number_of_cyclist_injured', 'number_of_cyclist_killed',
        'number_of_motorist_injured', 'number_of_motorist_killed',
        'contributing_factor_vehicle_1', 'collision_id', 'vehicle_type_code1',
        'borough', 'zip_code', 'vehicle_id', 'state_registration',
        'vehicle_type', 'contributing_factor_1', 'vehicle_occupants',
        'driver_sex', 'driver_license_status', 'pre_crash', 'point_of_impact',
        'public_property_damage', 'crash_date', 'crash_time', 'person_type',
        'person_injury', 'ped_role', 'person_sex', 'person_age',
        'emotional_status'],
       dtype='object')]

In [83]:
merged_df_cleaned2.head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
number_of_persons_injured,0,0,0,0,0,0,0,0,0,0
number_of_persons_killed,0,0,0,0,0,0,0,0,0,0
number_of_pedestrians_injured,0,0,0,0,0,0,0,0,0,0
number_of_pedestrians_killed,0,0,0,0,0,0,0,0,0,0
number_of_cyclist_injured,0,0,0,0,0,0,0,0,0,0
number_of_cyclist_killed,0,0,0,0,0,0,0,0,0,0
number_of_motorist_injured,0,0,0,0,0,0,0,0,0,0
number_of_motorist_killed,0,0,0,0,0,0,0,0,0,0
contributing_factor_vehicle_1,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,Passing or Lane Usage Improper,Driver Inexperience,Driver Inexperience,Unspecified,Alcohol Involvement
collision_id,4456314,4457022,4457022,4457022,4457022,4456686,4456588,4456588,4456879,4455749


In [84]:
#columns to drop:vehicle_id, state_registration, driver_license_status, public_property_damage, ped_role, emotional_status
#columns to rename: contributing_factor_vehicle 1; remove 1 , contributing_factor_1 --> contributing_factor_person

In [ ]:
#github test